In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from bs4 import BeautifulSoup
import requests
from yahooquery import Ticker


In [ ]:
#Import all companies ticker (B3)

url = 'https://www.tradingview.com/markets/stocks-brazil/market-movers-large-cap/'

soup = BeautifulSoup(requests.get(url).content, 'html.parser')
table = soup.find_all('table')[0] 

tickers_list = []
names_list = []

for row in table.tbody.find_all('tr'):
    ticker = row.find_all('td')[1].text.strip()
    name = row.find_all('td')[0].text.strip()[:5]
    tickers_list.append(name + ".SA")

print(tickers_list)

#CONTENT MISSING FROM YF
tickers_list = [ticker for ticker in tickers_list if ticker not in ['CXSE3.SA', 'AAURE.SA', 'ABEV3.SA', 'VIVT3.SA', 'BBSE3.SA', 'ITUB3.SA', 'BPAC3.SA', 'BBDC3.SA', 'TIMS3.SA', 'BBAS3.SA', 'SANB3.SA', 'CMIG3.SA', 'RDOR3.SA']]



In [ ]:

#Adding names
names_list = []
marketCap_list = []
for ticker in tickers_list:
    tick = Ticker(ticker)
    names_list.append(tick.quote_type[ticker]['longName'])
    marketCap_list.append(tick.summary_detail[ticker]['marketCap'])

#Joining names_list and tickers_list in a dataframe

df = pd.DataFrame(list(zip(names_list, tickers_list, marketCap_list)), columns =['Name', 'Ticker', 'MarketCap'])
print(df)

#Sorting by MarketCap

df = df.sort_values(by=['MarketCap'], ascending=False)

#Selecting the top 20 companies

df = df.head(20)

#Converting ticker_list to new tickers

tickers_list = df['Ticker'].tolist()


In [ ]:
#Getting todays stock price

today = datetime.date.today()

prices = {}
for ticker in tickers_list:
    tick = Ticker(ticker)
    prices[ticker] = tick.history(start=today - datetime.timedelta(days=2), end=today)['adjclose'][0]

df_price = pd.DataFrame(list(prices.items()), columns=['Ticker', 'Price'])

#Left joining data and df by Ticker

dfJoin = pd.merge(df, df_price, on='Ticker', how='left')
dfJoin = dfJoin.dropna()

print(dfJoin)


In [ ]:
stock = Ticker(tickers_list[0]).balance_sheet()

totalDebt = []
totalAssets = []
avgDebtPayback = []

for ticker in tickers_list:
    try:    
        stock = Ticker(ticker).balance_sheet()
        totalDebt.append(stock.iloc[3]['TotalDebt'])
        totalAssets.append(stock.iloc[3]['TotalAssets'])
        avgDebtPayback.append((stock.iloc[3]['TotalDebt']-stock.iloc[3]['LongTermDebt'])/251)
    except:
        print(ticker)

dfJoin['TotalDebt'] = totalDebt
dfJoin['TotalAssets'] = totalAssets 
dfJoin['AvgDebtPayback'] = avgDebtPayback
dfJoin

In [ ]:
#Based on Selic rate

riskFreeRate = 0.1202

In [ ]:
#Volatilidade histórica

stockPrice_6m = []
dfs = []
for ticker in tickers_list:

    stockPrice_6m.append(Ticker(ticker).history(start=today - datetime.timedelta(days=180), end=today))

df = pd.concat(stockPrice_6m)
df.reset_index(inplace=True)

df

In [ ]:
#Daily returns

df['DailyReturn 6m'] = df['adjclose'].pct_change()

#Annualized volatility per ticker (252 trading days)

dfVol = df.groupby('symbol').agg({'DailyReturn 6m': ['std', 'mean']})
dfVol.columns = ['Volatility 6m', 'DailyReturn 6m']

dfVol['Volatility 6m'] = dfVol['Volatility 6m'] * np.sqrt(252)

#Rename column symbol to Ticker

dfVol = dfVol.reset_index()
dfVol = dfVol.rename(columns={'symbol': 'Ticker'})

#Joining dfJoin and dfVol

dfJoin = pd.merge(dfJoin, dfVol, on='Ticker', how='left')

dfJoin

In [ ]:
#Options scraping

import cloudscraper
import json

url = "https://statusinvest.com.br/opcoes/" 

options = []
assets = []
expirations = []
for ticker in tickers_list:
    try:
        scraper = cloudscraper.create_scraper()
        res = scraper.get(url + ticker[:5]).text

        soup = BeautifulSoup(res, 'html.parser')
        
        lists = soup.find_all("input", attrs={'name': lambda name: name and name == 'options'})
        for l in lists:
            expiration = l.parent.parent.findChildren("h3", recursive=False)[0].findChildren("span", recursive=False)[0].text
            
            json_list = json.loads(l.attrs['value'])
            opt = pd.DataFrame.from_records(json_list)
            opt['Expiration'] = expiration  # Add 'Expiration' column with the corresponding expiration
            assets.append(opt.loc[0]['Asset'])
            options.append(opt)
    except:
        print(ticker)


    options

In [ ]:
#Creating a new data frame with only the Tickers and Price

dfPrice = dfJoin[['Ticker', 'Price', 'Volatility 6m']]

#Selecting only the first 4 letters of the ticker

dfPrice['Ticker'] = dfPrice['Ticker'].str[:5]

#Joining dfPrice and options

dfOptions = pd.merge(dfPrice, pd.concat(options), left_on='Ticker', right_on = 'Asset', how='left')

#Droping asset column

dfOptions = dfOptions.drop(columns=['Asset'])

#Changing Price_x to Stock Price

dfOptions = dfOptions.rename(columns={'Price_x': 'Stock Price'})
dfOptions = dfOptions.rename(columns={'Price_y': 'Option Price'})

dfOptions

In [ ]:
# Assuming you have a DataFrame called dfOptions and a column named 'Expiration'

dfOptions['Expiration'] = pd.to_datetime(dfOptions['Expiration'], format='%d/%m/%Y')

# Convert 'today' to a DatetimeArray object
today = pd.to_datetime(datetime.date.today())

dfOptions['Time to Expiration'] = (dfOptions['Expiration'] - today).dt.days

dfOptions

In [ ]:
import scipy.stats as si
def BSM_call(S, K, T, r, sigma):
    # S: spot price
    # K: strike price
    # T: time to maturity
    # r: interest rate
    # sigma: volatility of underlying asset
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = (np.log(S / K) + (r - 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    call = (S * si.norm.cdf(d1, 0.0, 1.0) - K * np.exp(-r * T) * si.norm.cdf(d2, 0.0, 1.0))
    return call


In [ ]:
def vega(S, K, T, r, sigma):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    vega = S * si.norm.pdf(d1, 0.0, 1.0) * np.sqrt(T)
    return vega


In [ ]:
def newtons_method(S, K, T, r, C, sigma_est, iterations):
    # S: spot price
    # K: strike price
    # T: time to maturity
    # r: interest rate
    # sigma_est: initial volatility estimate
    # iterations: number of iterations
    for i in range(iterations):
        sigma_est -= ((BSM_call(S, K, T, r, sigma_est) - C)
                      / vega(S, K, T, r, sigma_est))
    return sigma_est


In [ ]:
import scipy.stats as si

def implied_vol(df):
    # initial guess
    sigma_est = df['Volatility 6m']
    S = df['Stock Price']
    K = df['Strike']
    T = df['Time to Expiration'] / 365.0
    r = riskFreeRate
    C = df['Option Price']
    
    try:
        result = newtons_method(S, K, T, r, C, sigma_est, 100)
        return result
    except:
        return np.nan

dfOptions['ImpliedVolatility'] = dfOptions.apply(implied_vol, axis=1)
dfOptions = dfOptions.dropna()


In [ ]:
#Default options
from scipy.stats import norm
d1_list = []
def d1(df):
    sigma_est = df['Volatility 6m']
    S = df['Stock Price']
    K = df['Strike']
    T = df['Time to Expiration'] / 365.0
    r = riskFreeRate
    C = df['Option Price']    

    d1 = (np.log(S / K) + (r + 0.5 * sigma_est ** 2) * T) / (sigma_est * np.sqrt(T))

    return d1

dfOptions['Default'] = dfOptions.apply(d1, axis=1)

dfOptions['Default'] = norm.cdf(dfOptions['Default'])

dfOptions

In [ ]:
import numpy as np
from scipy.stats import norm

def black_scholes(total_assets, total_debt, volatility, risk_free_rate, time_to_maturity):
    d1 = (np.log(total_assets / total_debt) + (risk_free_rate + 0.5 * volatility ** 2) * time_to_maturity) / (volatility * np.sqrt(time_to_maturity))
    d2 = d1 - volatility * np.sqrt(time_to_maturity)
    
    market_value_debt = total_debt * norm.cdf(-d2) - total_assets * np.exp(-risk_free_rate * time_to_maturity) * norm.cdf(-d1)
    return market_value_debt

#Input the time to maturity in days

timeMaturity =  input("Enter the time to maturity in days: ")
dfJoin['Market Value Debt'] = dfJoin.apply(lambda x: black_scholes(x['TotalAssets'], x['TotalDebt'], x['Volatility 6m'], riskFreeRate, int(timeMaturity)/365), axis=1)
dfJoin

In [ ]:
#Default using dfJoin

def default_probability(V, D, r, sigma, T):
    d2 = (np.log(V / D) + (r - 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    return norm.cdf(-d2)

dfJoin["defaultProb"] = dfJoin.apply(lambda x: default_probability(x['TotalAssets'], x['TotalDebt'], riskFreeRate, x['Volatility 6m'], int(timeMaturity)/365), axis=1)

dfJoin